# Preparando el dataset
Utilizaremos FairFace como dataset. Este posee las caracteristicas de que tiene igual proporcion en todas las etnias, a saber, Medio Oriente, Blancos, Negros, Hispanos-Latinos, Indios, Sur de Asia y Este de Asia. Al tener una buena proporcion de las etnias que generalmente no estan correctamente representadas no existira un bias significativo en el modelo. Las fotos han sido cropped por lo que este paso no es necesario. Debemos tener en cuenta que de procesar fotos nuevas estas tambien tiene que ser cropped utilizando dlib o alguna otra herramienta similar.

No nos hara falta las fotos del Medio Oriente por lo que podemos desecharlas.

In [1]:
import pandas

train_csv = pandas.read_csv('./Datasets/FairFace/fairface_label_train.csv')
test_csv = pandas.read_csv('./Datasets/FairFace/fairface_label_val.csv')

train_csv = train_csv[train_csv['race'] != 'Middle Eastern']
test_csv = test_csv[test_csv['race'] != 'Middle Eastern']

Mezclamos las dos etnias asiaticas en una, ya que no se nos exige distinguirlas.

In [2]:
indexs_train = train_csv[(train_csv['race'] == 'East Asian') | (train_csv['race'] == 'Southeast Asian')].index

train_csv.loc[indexs_train, 'race'] = 'Asian'
 
indexs_test = test_csv[(test_csv['race'] == 'East Asian') | (test_csv['race'] == 'Southeast Asian')].index

test_csv.loc[indexs_test, 'race'] = 'Asian'

# Convirtiendo las imagenes a feature vectors
Una vez tenemos el dataset preparado procedemos a convertir las imagenes a vectores de caracteristicas que sirvan de input para la red. Se carga la imagen en rgb y se resize a la size que se pide, se transforma esta en una matriz de 21x28 con vectores de longitud 3 por casilla, indicando cada valor en este vector el valor correspondiente al pixel en cada uno de los colores. Posteriormente se normalizan.

In [3]:
import keras.preprocessing.image as image

def getPixels(file):
    img = image.load_img(file, target_size=(21, 28,))
    return image.img_to_array(img)

train_features = []
for _, value in train_csv['file'].items():
    train_features.append(getPixels('./Datasets/FairFace/' + value)/255)

test_features = []
for _, value in test_csv['file'].items():
    test_features.append(getPixels('./Datasets/FairFace/' + value)/255)

# Labels encoding
Una vez listos los features vectors debemos codificar los labels en vectores one hot para el entrenamiento.

In [4]:
labels_categories = {}

for index, item in enumerate(train_csv['race'].unique()):
    labels_categories[item] = index

train_labels = []
test_labels = []

for _, item in train_csv['race'].items():
    train_labels.append(labels_categories[item])

for _, item in test_csv['race'].items():
    test_labels.append(labels_categories[item])

import keras.utils as utils

one_hot_train = utils.to_categorical(train_labels, num_classes=5)
one_hot_test = utils.to_categorical(test_labels, num_classes=5)

# Creacion del train set y del validation set
Teniendo el train set y el test set, ambos preparados con los feature vectors correspondientes, se procede a la creacion del set de validacion. Para ello nos guiamos por lo pedido en la orden y usamos el 80% para el train set, hacemos shuffle para randomizar el orden.

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score

train_set, val_set, train_set_labels, val_set_labels = train_test_split(train_features, one_hot_train, train_size=0.80, shuffle=True)

# Creacion de la red convolucional
Una vez tengamos todo nuestro dataset procesado y listo podemos hacerle fit a la red. Pero primero hay que hacer una red :). Evidentemente los primero que pensamos es una convolucional puesto son las mejores en cuanto a clasificacion de imagenes. La idea obviamente va a ser probar con varias arquitecturas y parametros de las mismas para buscar la de mejor comportamiento.

In [6]:
from keras.models import Sequential, load_model
from keras import regularizers
from keras.layers import Conv2D, Flatten, Dense, Dropout
import numpy as np

In [ ]:
# intentando crear vgg-face mas nunca esto termina
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(21, 28, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))



In [28]:
# convolucional simple
model = Sequential()

model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(21, 28, 3), kernel_regularizer=regularizers.l2(0.01)))
model.add(Conv2D(128, kernel_size=3, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(5, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(np.array(train_set), np.array(train_set_labels), validation_data=(np.array(val_set), np.array(val_set_labels),), epochs=10, verbose=1)
# no muy buena precision, necesitamos mas overfitting :)
model.save('cnn_simple_model1.h5')

Epoch 1/10
1939/1939 [==============================] - 855s 412ms/step - loss: 1.6505 - accuracy: 0.4080 - val_loss: 1.2271 - val_accuracy: 0.5172
Epoch 2/10
1939/1939 [==============================] - 772s 397ms/step - loss: 1.1747 - accuracy: 0.5401 - val_loss: 1.1543 - val_accuracy: 0.5529
Epoch 3/10
1939/1939 [==============================] - 806s 416ms/step - loss: 1.0772 - accuracy: 0.5877 - val_loss: 1.1170 - val_accuracy: 0.5704
Epoch 4/10
1939/1939 [==============================] - 808s 417ms/step - loss: 0.9952 - accuracy: 0.6241 - val_loss: 1.1119 - val_accuracy: 0.5744
Epoch 5/10
1939/1939 [==============================] - 805s 415ms/step - loss: 0.9051 - accuracy: 0.6673 - val_loss: 1.1448 - val_accuracy: 0.5771
Epoch 6/10
1939/1939 [==============================] - 718s 370ms/step - loss: 0.8076 - accuracy: 0.7114 - val_loss: 1.1142 - val_accuracy: 0.5915
Epoch 7/10
1939/1939 [==============================] - 782s 403ms/step - loss: 0.7096 - accuracy: 0.7546 - val_

# Carga de los modelos
Se carga el modelo que se desee.

In [ ]:
model = load_model('./cnn_simple_model1.h5')

# Test de los modelos
Una vez el modelo en memoria se procede a probar su presicion en el test set.

In [8]:
predict_prob = model.predict(np.array(test_features))

def tranToOneHotPredicted(predicted: np.array):
    my_list = []
    len_vectors = predicted[0].shape[0]
    for index in range(predicted.shape[0]):
        array = np.zeros((len_vectors,))
        array[np.argmax(predicted[index])] = 1
        my_list.append(array)
    return np.array(my_list)

predictions = tranToOneHotPredicted(predict_prob)

print(precision_score(np.array(one_hot_test), predictions, average='weighted'))

0.579210863367728
